# Machine Learning :

Implement a suitable classical machine learning algorithm to detect characters in these signs and display detected
text finally.


Algorithm:

Step 1: Read Image using Opencv

Step 2: Extract text from the image using Opencv Methods

Step 3: Extract the character using masking method

Step 4: Comparing the ground-truth and extracted image by using cosine similarity
    

In [1]:
# Dependencies
import numpy as np
import cv2
from imutils import contours as sort_contour
from scipy import spatial
import glob
import os
import pandas as pd



In [7]:
def similarity(ground_truth,predict):
    
    '''Comparing collect images for character with extracted images"'''
    reshape_size=(50,50)
    x= cv2.resize(ground_truth,reshape_size,interpolation = cv2.INTER_LINEAR)
    y= cv2.resize(predict,reshape_size,interpolation = cv2.INTER_LINEAR)
    x,y= x.flatten()/255,y.flatten()/255  #normalization
    sim = -1 * (spatial.distance.cosine(x, y) - 1)  #gives the cosine value for both vectors
    return sim


def character_extractor(word, path_name='NA'):
    word_c=word.copy()
    if word.shape[0]==0 or word.shape[1]==0:return word_c,{}
    
    Alphabets="ABCDEFGHIJKLMNOPQRSTUVWXYZ_+"
#     word=adjust_gamma(word)
    mask=masking(word,np.array([0, 0, 164]),np.array([255, 255, 255]))
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) # finding contours for masked image
    if contours:
        contours, _ = sort_contour.sort_contours(contours, method="left-to-right")
    roi_count=0
    dic={}
    for c in contours:
        dic['ROI'+str(roi_count)]={}
        x,y,w,h = cv2.boundingRect(c)
        predict=mask[y:y + h,x:x + w]
         
        if write:
            path='./prepare_dataset/{}/roi_{}.jpg'.format(path_name,roi_count)
            cv2.imwrite(path,predict)
        else:    
            for ch in Alphabets:
                for ch_ls in glob.glob('./Character_Data/{}/*.jpg'.format(ch)):
                    ground_truth=cv2.imread(ch_ls,0)
                    sim= similarity(ground_truth,predict) 
                    '''0.97 is the threshold value given. If the cosine value is greater or equal to threshold value then
                        the both images having similarity greater and equal to 97%.'''
                    if sim >=0.97: 
                        dic['ROI'+str(roi_count)][ch]=sim     

            cv2.rectangle(word_c, (x, y), (x + w, y + h), (0,0,255), 1)
        
        roi_count+=1 
    return word_c,dic 

def adjust_gamma(image, gamma=1.1):
    '''Used to adjust the brightness or gamma value of given image'''
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)


def masking(img,lower, upper):
    '''Given image is masked to lower and upper value that is passed in the function. The lower and Upper
       are extracted by using HSV values'''
#     print(img.shape,"masking")
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)             
    mask = cv2.inRange(imgHSV,lower,upper)
    return mask


def preprocess(img,thresh_value=[145 ,145 ,145],name="NA"):
    global write
#     print(img.shape,"preprocess")
    
    img_ori=img.copy()
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    img = cv2.filter2D(img, -1, sharpen_kernel)
    white=[255,255,255]
    blue=[255,0,0]
    # finding the color and coverting the text color into blue
    for y in range(0,img.shape[0]):
        for x in range(0,img.shape[1]):
            if img[y,x,0]>=thresh_value[0] and img[y,x,1]>thresh_value[1] and img[y,x,2]>thresh_value[2]:img[y,x] = blue
            else:img[y,x] = white 

    mask=masking(img,np.array([0,0,0]),np.array([255,14,255]))
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    mask = cv2.erode(mask,kernel,iterations = 1) #erosion
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) #to extract the text from mask image
    if contours:
        contours, _ = sort_contour.sort_contours(contours, method="left-to-right")
    
    text_dic={}
    text_count=0
    for c in contours:
        area = cv2.contourArea(c)
        if (float(area)/float(mask.shape[0]*mask.shape[1]) > 0.02):
            x,y,w,h = cv2.boundingRect(c)
            if float(h)/float(mask.shape[0]) > 0.2 and float(h)/float(mask.shape[0]) < 0.7:
                if write: #please make this variable 'True' if you want to save the data
                    try:os.mkdir('./prepare_dataset/{}/{}'.format(name,text_count))
                    except:pass 
                    img_ori[y:y + h,x:x + w],result=character_extractor(img_ori[y:y + h,x:x + w],path_name="{}/{}".format(name,text_count))
                else:img_ori[y:y + h,x:x + w],result=character_extractor(img_ori[y:y + h,x:x + w])
                text_dic["Text"+str(text_count)]=result
                cv2.rectangle(img_ori, (x, y), (x + w, y + h), (0,255,0), 1)
                text_count+=1
    
    return img_ori,mask,text_dic
    
    
write=False    
img=cv2.imread('./TestSet/2.png')
h,w=img.shape[:2]
img,mask,results=preprocess(img)

print("Results",results)


cv2.imshow("img1",img)
cv2.imshow("mask",mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

        

Results {'Text0': {'ROI0': {'R': 0.999992952672644}, 'ROI1': {'_': 0.9842400830772055}, 'ROI2': {'E': 0.9999860599222188}, 'ROI3': {'S': 0.9999867589352665}, 'ROI4': {'T': 0.9999859038704109}}, 'Text1': {'ROI0': {}}, 'Text2': {'ROI0': {'M': 0.999994053100458}, 'ROI1': {'I': 0.9999930824283155, 'L': 0.9742207647254794, '_': 0.9742207647254794}, 'ROI2': {'L': 0.9999956937060123}, 'ROI3': {'E': 0.999991922036403}}, 'Text3': {'ROI0': {'+': 0.9999915388987497}, 'ROI1': {'_': 0.9886872400809098}, 'ROI2': {'+': 0.9999908160801005}}}


# Prepare Dataset

Make Character A-Z,1-9 Folders

In [ ]:
Alphanum="1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ_+"
# print(len(Alphabets))

for i in Alphanum:
    try:os.mkdir('./Character_Data/{}'.format(i))
    except:pass    

In [ ]:
thres=[[115 ,115 ,115],[145,145 ,145],[100 ,100 ,100],[150 ,150 ,150]] #threshold used to convert the text into blue 


write=True #to save the binary character images in respected folders
for itr_thres in thres:
    try:os.mkdir('./prepare_dataset/{}'.format(str(itr_thres)))
    except:pass 
    for each_img in glob.glob('./TestSet/*.png'):
        fol_1=each_img.split('\\')[-1].replace(".","_")
        try:os.mkdir('./prepare_dataset/{}/{}'.format(str(itr_thres),fol_1))
        except:pass    
        img=cv2.imread(each_img)
        img,mask,results[str(itr_thres)]=preprocess(img,name="{}/{}".format(str(itr_thres),fol_1), thresh_value=itr_thres)
#         cv2.imshow("img1",img)
#         cv2.imshow("mask",mask)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        
            


# Saving to excel

In [4]:
def analytic_txt(result_dic):
    '''Find the maximum character extracted from the images.'''
    text_analytics={}
    final_res=[]
    for thres_val,attr1 in result_dic.items():
        for txt,attr2 in attr1.items():  
            text_analytics[txt]=''
            for roi_, attr3 in attr2.items():   
                for wd in attr3.keys():
                    text_analytics[txt]+=wd
        final_res.append(text_analytics)  

    lenght_ls=[]    
    for len_ in final_res:
        lenght_ls.append(len(len_.keys()))
    select_text=final_res[lenght_ls.index(max(lenght_ls))]   
    final_text=''
    for ww in select_text.values():
        final_text+=ww

    return final_text     
    

In [21]:
write=False #Made into True only to collect the dataset
data=[]

color_thres= [[115 ,115 ,115],[145,145 ,145],[100 ,100 ,100],[150 ,150 ,150]]
for each_img in glob.glob('./TestSet/*.png'):
    results={}
    for itr_thres in color_thres:
        img=cv2.imread(each_img)
        img,mask,results[str(itr_thres)]=preprocess(img, thresh_value=itr_thres)
    data.append([each_img.split('\\')[-1],analytic_txt(results)])
#         cv2.imshow("img1",img)
#         cv2.imshow("mask",mask)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [22]:
data

[['1.png', '_IL_IL___IL_IL_IL_IL_IL__IL_IL_IL__'],
 ['10.png', 'IL_IL_IL_'],
 ['11.png', '+___'],
 ['12.png', '+_L____L__'],
 ['13.png', '__IL_____IL_+SI_NG_E__N'],
 ['14.png', 'B_A_B__+IL_IL_'],
 ['15.png', ''],
 ['16.png', ''],
 ['17.png', ''],
 ['18.png', ''],
 ['19.png', ''],
 ['2.png', 'R_ESTMIL_LE+_+'],
 ['20.png', ''],
 ['21.png', ''],
 ['22.png', 'L_IL_'],
 ['23.png', ''],
 ['24.png', ''],
 ['25.png', ''],
 ['3.png', ''],
 ['4.png', 'IL_'],
 ['5.png', '_IL__'],
 ['6.png', '_'],
 ['7.png', 'IL_IL_IL_IL__IL_IL_IL_IL_IL__'],
 ['8.png', ''],
 ['9.png', 'N+_EL_U+_IL____']]

In [ ]:
df=pd.DataFrame(data,columns=['file_name','text'])
print(df.head())

df.to_excel("submission.xlsx") #dataframe to excel

Problem Faced:

1. Image Quality were affecting as each quality will be having different threshold values

2. Charcter colors like black and white character

3. Postion or angle of the sign-board

4. Dimensions of the images

Note: "+" folder can further used to extract the each character by using Dilation method.